In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import numpy as np
import datetime
import pickle

from collections import OrderedDict

from models import Generator, predict_many, predict_one
from data import load_data_from_pickle, translate

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.get_device_name(), "|", torch.cuda.is_available())

# Data Loading

In [ ]:
dataset_name = "rockyou"
charmap, inv_charmap = load_data_from_pickle(dataset_name, train_data=False, test_data=False)

# Predictions

In [ ]:
checkpoints_to_predict = OrderedDict()
checkpoints_to_predict['1'] = "Checkpoints/netG103:55:21AM_12-03-20"
checkpoints_to_predict['1000'] = "Checkpoints/netG103:55:21AM_12-03-20"
checkpoints_to_predict['3000'] = "Checkpoints/netG103:55:21AM_12-03-20"
checkpoints_to_predict['5000'] = "Checkpoints/netG103:55:21AM_12-03-20"
checkpoints_to_predict['7000'] = "Checkpoints/netG103:55:21AM_12-03-20"
checkpoints_to_predict['9000'] = "Checkpoints/netG103:55:21AM_12-03-20"

In [ ]:
for key, value in checkpoints_to_predict.items():
    netG = Generator(charmap).to(device)
    netG.load_state_dict(torch.load(value))
    print(predict_one(netG, 10))

# Old

In [ ]:
netG1 = Generator(charmap).to(device)
netG1.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG103:55:21AM_12-03-20"))

netG1000 = Generator(charmap).to(device)
netG1000.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG100004:06:51AM_12-03-20"))

netG3000 = Generator(charmap).to(device)
netG3000.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG300004:29:10AM_12-03-20"))

netG5000 = Generator(charmap).to(device)
netG5000.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG500004:51:18AM_12-03-20"))

netG7000 = Generator(charmap).to(device)
netG7000.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG700005:13:29AM_12-03-20"))

netG9000 = Generator(charmap).to(device)
netG9000.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG900005:35:43AM_12-03-20"))

In [ ]:
preds = predict_many([netG1, netG1000, netG3000, netG5000, netG7000, netG9000])
iters = ["1", "1000", "3000", "5000", "7000", "9000"]
for i, pred in zip(iters, preds):
    print(f"{i} iterations: {pred}")

In [ ]:
predict_one(netG1000, 100)